In [ ]:
!pip install segmentation-models-pytorch==0.3.3 albumentations==1.3.1 -q
!rsync -ah --progress "/content/drive/MyDrive/Kuliah/Skripsi S1/train/" "/content/train_local/"
!du -sh /content/drive/MyDrive/Kuliah/Skripsi\ S1/train

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.9 MB/s eta 0:00:00


In [ ]:
import os
import random
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
IMAGES_DIR = "/content/drive/MyDrive/Kuliah/Skripsi S1/train/images"
MASKS_DIR  = "/content/drive/MyDrive/Kuliah/Skripsi S1/train/targets"

IMG_SIZE   = 512
BATCH_SIZE = 8
NUM_EPOCHS = 20
LR         = 1e-4
VAL_SPLIT  = 0.1

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

Using device: cuda


In [ ]:
class BuildingDataset(Dataset):
    def __init__(self, image_ids, images_dir, masks_dir, transform=None):
        self.image_ids = image_ids
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        img_path = os.path.join(self.images_dir, image_id)

        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if image is None:
            raise RuntimeError(f"Cannot read image: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        base = image_id
        if base.endswith(".png"):
            base = base[:-4]

        base_clean = base.replace(" (1)", "")

        mask_name = base_clean + "_target.png"
        mask_path = os.path.join(self.masks_dir, mask_name)

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"Fail to read mask, using empty mask: {mask_path}")
            h, w = image.shape[:2]
            mask = np.zeros((h, w), dtype="uint8")

        mask = (mask > 0).astype("float32")

        if self.transform is not None:
            augmented = self.transform(image=image, mask=mask)
            image = augmented["image"]
            mask  = augmented["mask"].unsqueeze(0)
        else:
            image = ToTensorV2()(image=image)["image"]
            mask  = torch.from_numpy(mask).unsqueeze(0)

        return image, mask

In [ ]:
all_images = sorted(
    [
        f for f in os.listdir(IMAGES_DIR)
        if os.path.isfile(os.path.join(IMAGES_DIR, f))
        and f.endswith("_pre_disaster.png")
    ]
)

print(f"Total pre-disaster images found: {len(all_images)}")

random.seed(42)
random.shuffle(all_images)

n_total = len(all_images)
n_val = int(n_total * VAL_SPLIT)
val_ids = all_images[:n_val]
train_ids = all_images[n_val:]

print(f"Train: {len(train_ids)}, Val: {len(val_ids)}")


Total pre-disaster images found: 2799
Train: 2240, Val: 559


In [ ]:
train_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

train_dataset = BuildingDataset(train_ids, IMAGES_DIR, MASKS_DIR, transform=train_transform)
val_dataset   = BuildingDataset(val_ids,   IMAGES_DIR, MASKS_DIR, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=8)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=8)


In [ ]:
model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
)

model = model.to(DEVICE)
print("Model parameters (M):", sum(p.numel() for p in model.parameters()) / 1e6)

Model parameters (M): 14.328209


In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, eps=1e-7):
        super().__init__()
        self.eps = eps

    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)
        dims = (0, 2, 3)
        intersection = torch.sum(probs * targets, dims)
        union = torch.sum(probs, dims) + torch.sum(targets, dims)
        dice = (2 * intersection + self.eps) / (union + self.eps)
        return 1 - dice.mean()

# region metrics

def iou_score_from_logits(logits, targets, threshold=0.5, eps=1e-7):
    probs = torch.sigmoid(logits)
    preds = (probs > threshold).float()

    intersection = torch.sum(preds * targets)
    union = torch.sum(preds) + torch.sum(targets) - intersection
    iou = (intersection + eps) / (union + eps)
    return iou.item()

def dice_score_from_logits(logits, targets, threshold=0.5, eps=1e-7):
    probs = torch.sigmoid(logits)
    preds = (probs > threshold).float()

    intersection = torch.sum(preds * targets)
    union = torch.sum(preds) + torch.sum(targets)
    dice = (2 * intersection + eps) / (union + eps)
    return dice.item()


# def boundary_map(mask):
#     """
#     Approximate boundary map from binary mask (B,1,H,W) using morphological erosion.
#     Returns a float tensor in [0,1] with same shape.
#     """
#     # mask: float 0/1
#     # 3x3 erosion
#     kernel = torch.ones((1, 1, 3, 3), device=mask.device)
#     eroded = F.conv2d(mask, kernel, padding=1)
#     eroded = (eroded == 9).float()  # only keep pixels where all 9 neighbors are 1
#     boundary = mask - eroded
#     boundary = (boundary > 0).float()
#     return boundary

# def boundary_f_score_from_logits(logits, targets, threshold=0.5, dilation_ratio=0.02, eps=1e-7):
#     """
#     Compute boundary F-score between predicted and target masks.
#     Implementation: Kokkinos-style tolerant boundary F.
#     """
#     probs = torch.sigmoid(logits)
#     preds = (probs > threshold).float()        # (B,1,H,W)
#     gts   = (targets > 0.5).float()

#     B, _, H, W = preds.shape
#     # tolerance in pixels
#     tol = max(1, int(round(dilation_ratio * max(H, W))))

#     # get boundaries
#     pred_b = boundary_map(preds)
#     gt_b   = boundary_map(gts)

#     if tol > 1:
#         # dilate boundaries using max-pool
#         pred_dil = F.max_pool2d(pred_b, kernel_size=2*tol+1, stride=1, padding=tol)
#         gt_dil   = F.max_pool2d(gt_b,   kernel_size=2*tol+1, stride=1, padding=tol)
#     else:
#         pred_dil = pred_b
#         gt_dil   = gt_b

#     # precision: how many predicted boundary pixels match gt boundary (within tol)
#     # recall: how many gt boundary pixels are matched by predicted boundary
#     pred_match = pred_b * gt_dil
#     gt_match   = gt_b * pred_dil

#     # sum across batch
#     pred_b_sum = pred_b.sum()
#     gt_b_sum   = gt_b.sum()
#     pred_match_sum = pred_match.sum()
#     gt_match_sum   = gt_match.sum()

#     if pred_b_sum == 0 and gt_b_sum == 0:
#         return 1.0  # no boundaries in either → perfect
#     if pred_b_sum == 0 or gt_b_sum == 0:
#         return 0.0  # one has boundary, the other doesn't

#     precision = (pred_match_sum + eps) / (pred_b_sum + eps)
#     recall    = (gt_match_sum   + eps) / (gt_b_sum   + eps)
#     bf = (2 * precision * recall + eps) / (precision + recall + eps)
#     return bf.item()

# ---- loss & optimizer ----

bce_loss  = nn.BCEWithLogitsLoss()
dice_loss = DiceLoss()

def seg_loss(logits, targets):
    return 0.5 * bce_loss(logits, targets) + 0.5 * dice_loss(logits, targets)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)


In [ ]:

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    train_loss = 0.0
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch} - Training", leave=False)

    for step, (images, masks) in enumerate(train_loader_tqdm, start=1):
        images = images.to(DEVICE)
        masks  = masks.to(DEVICE)

        optimizer.zero_grad()
        logits = model(images)
        loss   = seg_loss(logits, masks)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

        if step % 10 == 0:
            train_loader_tqdm.set_postfix({"loss": f"{loss.item():.4f}"})

    train_loss /= len(train_loader.dataset)

    # val
    model.eval()
    val_loss = 0.0
    val_iou  = 0.0
    val_dice = 0.0
    n_batches = 0

    val_loader_tqdm = tqdm(val_loader, desc=f"Epoch {epoch} - Validation", leave=False)

    with torch.no_grad():
        for images, masks in val_loader_tqdm:
            images = images.to(DEVICE)
            masks  = masks.to(DEVICE)

            logits = model(images)
            loss   = seg_loss(logits, masks)
            val_loss += loss.item() * images.size(0)

            # metrics
            batch_iou  = iou_score_from_logits(logits, masks)
            batch_dice = dice_score_from_logits(logits, masks)

            val_iou  += batch_iou
            val_dice += batch_dice
            n_batches += 1

            if n_batches % 5 == 0:
                val_loader_tqdm.set_postfix(
                    iou=f"{batch_iou:.4f}",
                    dice=f"{batch_dice:.4f}"
                )

    val_loss /= len(val_loader.dataset)
    val_iou  /= max(n_batches, 1)
    val_dice /= max(n_batches, 1)

    print(
        f"Epoch [{epoch}/{NUM_EPOCHS}] "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f} | "
        f"Val IoU: {val_iou:.4f} | "
        f"Val Dice: {val_dice:.4f}"
    )

    # ----- SAVE MODEL EVERY 5 EPOCHS -----
    if epoch % 5 == 0:
        save_path = f"/content/drive/MyDrive/Kuliah/Skripsi S1/checkpoints/UNet18_epoch{epoch}.pth"
        torch.save(model.state_dict(), save_path)
        print(f"Model saved: {save_path}")

Epoch 1 - Training:   0%|          | 0/280 [00:00<?, ?it/s]

Epoch 1 - Validation:   0%|          | 0/70 [00:00<?, ?it/s]

Epoch [1/20] Train Loss: 0.2800 | Val Loss: 0.2302 | Val IoU: 0.5947 | Val Dice: 0.7435


Epoch 2 - Training:   0%|          | 0/280 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7866a71bbf60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
     Exception ignored in:  <function _MultiProcessingDataLoaderIter.__del__ at 0x7866a71bbf60> 
^^Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
^    ^self._shutdown_workers()^
^^^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
^    ^if w.is_alive():^^

 Exception ignored in:    File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
 <function _MultiProcessingDataLoaderIter.__del__ at 0x7866a71bbf60> 
     Traceback (most rece

...

Epoch 19 - Validation:   0%|          | 0/70 [00:00<?, ?it/s]

Epoch [19/20] Train Loss: 0.1405 | Val Loss: 0.1378 | Val IoU: 0.6854 | Val Dice: 0.8119


Epoch 20 - Training:   0%|          | 0/280 [00:00<?, ?it/s]

Epoch 20 - Validation:   0%|          | 0/70 [00:00<?, ?it/s]

Epoch [20/20] Train Loss: 0.1382 | Val Loss: 0.1351 | Val IoU: 0.6894 | Val Dice: 0.8147
Model saved: /content/drive/MyDrive/Kuliah/Skripsi S1/checkpoints/UNet18_epoch20.pth


In [ ]:
model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights=None,
    in_channels=3,
    classes=1
).to(DEVICE)

# Load saved weights
save_path = "/content/drive/MyDrive/Kuliah/Skripsi S1/checkpoints/UNet18_epoch20.pth"
model.load_state_dict(torch.load(save_path, map_location=DEVICE))

print("Loaded model from:", save_path)


Loaded model from: /content/drive/MyDrive/Kuliah/Skripsi S1/checkpoints/UNet18_epoch20.pth


In [ ]:
import os
import cv2

TEST_IMAGES_DIR = "/content/drive/MyDrive/Kuliah/Skripsi S1/test/images"
TEST_MASKS_DIR  = "/content/drive/MyDrive/Kuliah/Skripsi S1/test/targets"

valid_test_ids = []

all_test_imgs = sorted(
    f for f in os.listdir(TEST_IMAGES_DIR)
    if os.path.isfile(os.path.join(TEST_IMAGES_DIR, f))
)

print("Total test images found:", len(all_test_imgs))

for fname in all_test_imgs:
    img_path = os.path.join(TEST_IMAGES_DIR, fname)

    # remove .png
    if not fname.endswith(".png"):
        continue
    base = fname[:-4]

    # handling weird duplicates:
    # 1) exact same base + "_target"
    # 2) base with " (1)" removed + "_target"
    base_clean = base.replace(" (1)", "")

    cand1 = os.path.join(TEST_MASKS_DIR, base + "_target.png")
    cand2 = os.path.join(TEST_MASKS_DIR, base_clean + "_target.png")

    mask_path = None
    if os.path.exists(cand1):
        mask_path = cand1
    elif os.path.exists(cand2):
        mask_path = cand2

    if mask_path is None:
        continue

    m = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if m is None:
        print("Unreadable mask, skipping:", mask_path)
        continue

    valid_test_ids.append(fname)

print("Valid test images with readable masks:", len(valid_test_ids))

Total test images found: 1867
Valid test images with readable masks: 1867


In [ ]:
test_dataset = BuildingDataset(
    valid_test_ids,
    TEST_IMAGES_DIR,
    TEST_MASKS_DIR,
    transform=val_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
model.eval()
test_iou  = 0.0
test_dice = 0.0
test_bf   = 0.0
n_batches = 0

with torch.no_grad():
    for images, masks in tqdm(test_loader, desc="Testing", leave=False):

        if masks is None:
          continue

        images = images.to(DEVICE)
        masks  = masks.to(DEVICE)

        logits = model(images)

        batch_iou  = iou_score_from_logits(logits, masks)
        batch_dice = dice_score_from_logits(logits, masks)

        test_iou  += batch_iou
        test_dice += batch_dice
        n_batches += 1

test_iou  /= max(n_batches, 1)
test_dice /= max(n_batches, 1)

print("=== TEST RESULTS ===")
print(f"Test IoU:  {test_iou:.4f}")
print(f"Test Dice: {test_dice:.4f}")


=== TEST RESULTS ===
Test IoU:  0.5451
Test Dice: 0.6928
Test BF:   0.8670


In [ ]:
ckpt_path = "/content/drive/MyDrive/Kuliah/Skripsi S1/checkpoints/UNet18_epoch20.pth"

model.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
model.to(DEVICE)
model.eval()

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [ ]:
img_path = "/content/drive/MyDrive/Kuliah/Skripsi S1/test/images/hurricane-florence_00000013_pre_disaster.png"
mask_path = "/content/drive/MyDrive/Kuliah/Skripsi S1/test/targets/hurricane-florence_00000013_pre_disaster_target.png"

image = cv2.imread(img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
gt_mask = (gt_mask > 0).astype(np.float32)


In [ ]:
image = cv2.imread(img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
mask = (mask > 0).astype("float32")

In [ ]:
val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])


In [ ]:
aug = val_transform(image=image, mask=mask)

image_tensor = aug["image"].unsqueeze(0).to(DEVICE)
mask_tensor  = aug["mask"].unsqueeze(0).unsqueeze(0).to(DEVICE)


In [ ]:
model.eval()
with torch.no_grad():
    logits = model(image_tensor)
    probs  = torch.sigmoid(logits)
    pred   = (probs > 0.5).float()

In [ ]:
iou  = iou_score_from_logits(logits, mask_tensor)
dice = dice_score_from_logits(logits, mask_tensor)

print(f"IoU:  {iou:.4f}")
print(f"Dice: {dice:.4f}")

IoU:  0.8075
Dice: 0.8935
BF:   0.9875
